## TO DO
+ Test:
MODEL_NAMES = {'baseline_lstm_v17':[None,600,550],'baseline_lstm_v15':[None],'baseline_lstm_v18':[None]}
with median ensemble
+ Test best model:
    with kernels free and median

In [ ]:
# MODEL_NAMES = {'baseline_lstm_v17':[None,600,550],'baseline_lstm_v15':[None],'baseline_lstm_v18':[None]}
# MODEL_NAMES = {'baseline_lstm_v17':[None,600,550],'baseline_lstm_v15':[None],'baseline_lstm_v18':[None,200]} 0.14210225998371193


In [1]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
from scipy.optimize import minimize
MODEL_PATH = '../03.SavedModels'
DATA_PATH  = '../01.Data/'
#MODEL_NAMES = {'baseline_lstm_v17':[None,550],'baseline_lstm_v15':[None]}
MODEL_NAMES = {'baseline_lstm_v17':[None,600,550],'baseline_lstm_v15':[None],'baseline_lstm_v18':[None,200]}
MODE_ENSEMBLE = 'median'
PREDICTION_ROUNDED = False
METHOD_ROUNDING = 2

In [2]:
def post_processing(df,press_min,press_step,press_max,col = 'pressure',method = 1):
    if method == 1:
        # ENSEMBLE FOLDS WITH MEDIAN AND ROUND PREDICTIONS
        df[col] =np.round( (df[col] - press_min)/press_step )* press_step + press_min
        df[col] = np.clip(df[col], press_min, press_max)
        return df
    elif method ==2:
        df[col] = df[col].apply(find_nearest)
        return df
    elif methos is None:
        return df
def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        return sorted_pressures[-1]
    elif insert_idx == 0:
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

def calculate_metric(df,pred = 'oof',target = 'pressure'):
    tmp = df.loc[df['u_out']==0].copy().reset_index(drop = True)
    return mean_absolute_error(tmp[pred],tmp[target])

In [3]:
%%time
train_df = pd.read_csv(os.path.join(DATA_PATH,'train_folds.csv'))
## Post Processing 1:
all_pressure = np.sort( train_df.pressure.unique() )
press_min    = all_pressure[0]
press_max    = all_pressure[-1]
press_step   = ( all_pressure[1] - all_pressure[0] )

# Post processing 2:
unique_pressures = train_df["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

CPU times: user 2.99 s, sys: 548 ms, total: 3.53 s
Wall time: 3.53 s


In [132]:
## =============================================
## Loading test predictions and OOF predictions
## =============================================

oof_list   = []
preds_list = []
oof_values = []
train_df = pd.read_csv('../01.Data/train_folds.csv')[['id','pressure','u_out']]
test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out']]

for name in tqdm(MODEL_NAMES.keys()):
    for checkpoint in MODEL_NAMES[name]:
        if checkpoint is not None:
            oof_df  = pd.read_csv(os.path.join(MODEL_PATH,name,f'oof_preds_epoch_{checkpoint}.csv')).sort_values(by = ['id'])
            pred_df = pd.read_csv(os.path.join(MODEL_PATH,name,f'test_preds_epoch_{checkpoint}_median.csv')).sort_values(by = ['id']) 
            oof_df = train_df.merge(oof_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            pred_df = test_df.merge(pred_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            if PREDICTION_ROUNDED:
                pred_df = post_processing(pred_df,press_min,press_step,press_max,col = 'preds',method = METHOD_ROUNDING)
        else:
            oof_df  = pd.read_csv(os.path.join(MODEL_PATH,name,'oof_preds.csv')).sort_values(by = ['id'])
            pred_df = pd.read_csv(os.path.join(MODEL_PATH,name,'test_preds.csv')).sort_values(by = ['id']) 
            oof_df = train_df.merge(oof_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            pred_df = test_df.merge(pred_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            if PREDICTION_ROUNDED:
                pred_df = post_processing(pred_df,press_min,press_step,press_max,col = 'preds',method = METHOD_ROUNDING)
            
        print(f"========== OOF METRIC: {name} - checkpoint[{checkpoint}] ========")
        mae_original = calculate_metric(oof_df,pred = 'oof',target = 'pressure')
        if PREDICTION_ROUNDED:
            oof_df = post_processing(oof_df,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
            mae_post = calculate_metric(oof_df,pred = 'oof',target = 'pressure')
            print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")
            oof_values.append(mae_post)
        else:
            print(f"MAE-ORIGIN:{mae_original}")
            oof_values.append(mae_original)
            
        oof_list.append(oof_df.oof.values) # With postprocessed values
        preds_list.append(pred_df.preds.values)
        del oof_df,pred_df

========== OOF METRIC: baseline_lstm_v17 - checkpoint[None] ========
MAE-ORIGIN:0.15254106207925197
========== OOF METRIC: baseline_lstm_v17 - checkpoint[600] ========
MAE-ORIGIN:0.153219068885379
========== OOF METRIC: baseline_lstm_v17 - checkpoint[550] ========
MAE-ORIGIN:0.15387579998538162
========== OOF METRIC: baseline_lstm_v15 - checkpoint[None] ========
MAE-ORIGIN:0.1611637197607971
========== OOF METRIC: baseline_lstm_v18 - checkpoint[None] ========
MAE-ORIGIN:0.15781139021428742
========== OOF METRIC: baseline_lstm_v18 - checkpoint[200] ========
MAE-ORIGIN:0.1584061861926046



## Ensembling with Scipy Optimize

In [133]:
indices = train_df[train_df['u_out']==0].index.values
X = np.stack(oof_list,1)[indices]
y = train_df['pressure'][indices]

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
# Check correct 
for oof_idx in range(X.shape[1]):
    print(f'OOF {oof_idx}: ',mean_absolute_error(X[:,oof_idx],y))    
    
x0 = np.array([1/X.shape[1]]*X.shape[1])
# The cost function is just the ensemble's mean absolute error.
def cost(x0):
    a = np.mean(np.abs((y - np.sum(X*x0, -1))))
    print(a)
    return a
bnds = tuple([(0,1)]*X.shape[1]) # Weights have to be between 0 and 1.
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)}) # Sum of the weights will be 1.
res = minimize(cost, x0, bounds=bnds,options = {'disp':True})
print(res)

X shape: (2290968, 6)
y shape: (2290968,)
OOF 0:  0.15254106207925197
OOF 1:  0.153219068885379
OOF 2:  0.15387579998538162
OOF 3:  0.1611637197607971
OOF 4:  0.15781139021428742
OOF 5:  0.1584061861926046
0.1459111927679058
0.14591119192913757
0.14591119193210592
0.14591119193823995
0.14591119197801608
0.14591119197040028
0.14591119197499322
8.597421701067612
8.597421877034986
8.597421877032192
8.597421877032621
8.597421877052351
8.59742187704223
8.597421877042663
0.14649754136864174
0.1464975621168641
0.1464975621191639
0.14649756212547022
0.14649756216658993
0.1464975621590926
0.14649756216339704
0.1459103474921488
0.14591034747340595
0.1459103474763163
0.14591034748247397
0.14591034752229068
0.14591034751464912
0.14591034751924806
0.14591034632537683
0.1459103463031931
0.14591034630610417
0.14591034631226157
0.14591034635207997
0.14591034634443656
0.1459103463490351
0.14591034231714697
0.14591034227360422
0.1459103422765162
0.14591034228266944
0.1459103423224835
0.14591034231483405

In [134]:
oof_final   = 0
preds_final = 0
print('Weights:',res.x)
for (oof,preds,w) in zip(oof_list,preds_list,res.x):
    oof_final   += oof*w
    preds_final += preds*w
    
train_df['oof'] = oof_final
tmp = train_df.loc[train_df['u_out']==0].reset_index(drop = True)
mae_original = mean_absolute_error(tmp.oof,tmp.pressure)
tmp = post_processing(tmp,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
mae_post = mean_absolute_error(tmp.oof,tmp.pressure)
print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")

Weights: [0.19379984 0.19059676 0.18382056 0.14001505 0.14842525 0.14336411]
MAE-ORIGIN:0.14576704441249194, MAE_POST: 0.1433893196827191


In [135]:
test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out']]
test_df['pressure'] = preds_final
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df[['id','pressure']].to_csv('submission_simple_ensemble.csv',index = False)

In [136]:
#1433

In [137]:
test_df[test_df['u_out']==0]['pressure'].mean()

17.643945276435186

In [138]:
train_df[train_df['u_out']==0]['pressure'].mean()

17.596211347845607

## Ensembling with Scipy Optimize by RC

In [139]:
train_optim = pd.read_csv('../01.Data/train_folds.csv')[['id','pressure','u_out','R','C']]
train_optim['RC'] = train_optim['R'].astype(str) + '_' +  train_optim['C'].astype(str)
test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out','R','C']]
test_df['RC'] = test_df['R'].astype(str) + '_' +  test_df['C'].astype(str)

'''
cols= [] 
for it in range(len(oof_list)):
    #train_optim[f'oof_{it+1}'] = oof_list[it]
    #test_df[f'preds_{it+1}'] = preds_list[it]
    cols.append(f'oof_{it+1}')
'''
    

"\ncols= [] \nfor it in range(len(oof_list)):\n    #train_optim[f'oof_{it+1}'] = oof_list[it]\n    #test_df[f'preds_{it+1}'] = preds_list[it]\n    cols.append(f'oof_{it+1}')\n"

In [140]:
weights = []
rc_list = []
def cost(x0):
    val = np.mean(np.abs((y - np.sum(X*x0, -1))))
    return val
for rc in train_optim['RC'].unique():
    indices = train_optim[(train_optim['u_out']==0)&(train_optim['RC']==rc)].index.values
    X = np.stack(oof_list,1)[indices]
    y = train_df['pressure'][indices]
    # The cost function is just the ensemble's mean absolute error.
    x0 = np.array([1/X.shape[1]]*X.shape[1])
    #x0[0] = 0.7
    bnds = tuple([(0,1)]*X.shape[1]) # Weights have to be between 0 and 1.
    cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)}) # Sum of the weights will be 1.
    res = minimize(cost, x0, bounds=bnds,options = {'disp':True})
    print(f'===================== RC: {rc}========================')
    rc_list.append((rc.split('_')[0],rc.split('_')[1]))
    print(res)
    weights.append(res.x)

===================== RC: 20_50========================
      fun: 0.13040852286602353
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00160843, -0.00206154, -0.00117646,  0.00098668,  0.001335  ,
        0.00108451])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 3
   status: 0
  success: True
        x: array([0.16667769, 0.16667775, 0.16667763, 0.16667734, 0.16667729,
       0.16667733])
===================== RC: 20_20========================
      fun: 0.13285028425089435
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([4.36628511e-04, 1.62625469e-04, 2.09068873e-04, 4.80643303e-05,
       3.66284780e-04, 3.68713393e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 259
      nit: 31
   status: 0
  success: True
        x: array([0.16312386, 0.22778813, 0.18828863, 0.11487266, 0.15072308,
       0.15523355])
===================== RC: 50_20=======================

In [141]:
for (R,C),w in zip(rc_list,weights):
    print(f'R:{R},C:{C}')
    ## Train
    indices = train_optim[(train_optim['R']==int(R))&((train_optim['C']==int(C)))].index.values
    train_optim.loc[indices,'oof'] = np.sum(np.stack(oof_list,1)[indices]*w,axis = -1)
    ## Test
    indices = test_df[(test_df['R']==int(R))&((test_df['C']==int(C)))].index.values
    test_df.loc[indices,'pressure'] = np.sum(np.stack(preds_list,1)[indices]*w,axis = -1)
    
    
tmp = train_optim.loc[train_optim['u_out']==0].reset_index(drop = True)
mae_original = mean_absolute_error(tmp.oof,tmp.pressure)
tmp = post_processing(tmp,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
mae_post = mean_absolute_error(tmp.oof,tmp.pressure)
print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")

R:20,C:50
R:20,C:20
R:50,C:20
R:50,C:50
R:5,C:50
R:5,C:20
R:50,C:10
R:20,C:10
R:5,C:10
MAE-ORIGIN:0.1458358469337877, MAE_POST: 0.14342896684144105


In [142]:
#1431

In [143]:
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_weight_optimization_by_rc.csv',index = False)

## Classical Ensembles

### 1. Mean / Median Ensemble:

In [144]:
print(oof_list)
print(preds_list)

[array([ 5.80932856,  5.85838032,  7.97244024, ..., 25.43886375,
       24.72276497, 21.76544571]), array([ 5.79564428,  5.84438467,  7.9414978 , ..., 25.68979454,
       24.93735695, 21.94672966]), array([ 5.80444765,  5.84198475,  7.91970968, ..., 25.97400665,
       24.96880341, 21.67961693]), array([ 5.78878021,  5.78226042,  7.96843767, ..., 21.47255707,
       22.44314766, 20.43738174]), array([ 5.78895521,  5.83629751,  7.99543142, ..., 20.99167824,
       22.38116074, 20.4294548 ]), array([ 5.78842354,  5.83482599,  8.00184345, ..., 20.99312401,
       22.37973404, 20.42835999])]
[array([ 6.2790966,  5.9550285,  7.129014 , ..., 15.029264 , 14.09877  ,
       12.393284 ]), array([ 6.2731366,  5.955427 ,  7.130601 , ..., 15.058527 , 14.026571 ,
       12.284483 ]), array([ 6.2774534,  5.943723 ,  7.1321554, ..., 15.104727 , 14.104841 ,
       12.4517765]), array([ 6.275116 ,  5.9496193,  7.1220026, ..., 16.366173 , 16.01205  ,
       15.254168 ]), array([ 6.264642 ,  5.963396 ,  

In [145]:
if MODE_ENSEMBLE == 'mean':
    oof_ens   = np.mean(np.vstack(oof_list),axis = 0)
    preds_ens = np.mean(np.vstack(preds_list),axis = 0)
    
elif MODE_ENSEMBLE == 'median':
    oof_ens   = np.median(np.vstack(oof_list),axis = 0)
    preds_ens = np.median(np.vstack(preds_list),axis = 0)

print(f"{'='*5} ENSEMBLE {MODE_ENSEMBLE} {'='*5}")

train_df['oof'] = oof_ens
tmp = train_df.loc[train_df['u_out']==0].reset_index(drop = True)
mae_original = mean_absolute_error(tmp.oof,tmp.pressure)
tmp = post_processing(tmp,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
mae_post = mean_absolute_error(tmp.oof,tmp.pressure)
print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")

===== ENSEMBLE median =====
MAE-ORIGIN:0.14448187883246522, MAE_POST: 0.14210225998371193


MODEL_NAMES = {'baseline_lstm_v17':[None],'baseline_lstm_v15':[None]}
+ Ensemble-Mean, prediction_rounded-False: 0.14574096082939075

MODEL_NAMES = {'baseline_lstm_v17':[None,600],'baseline_lstm_v15':[None]}
+ Ensemble-Mean, prediction_rounded-False: 0.144987971680227
+ Ensemble-Median, prediction_rounded-False: 0.1482044855214965

MODEL_NAMES = {'baseline_lstm_v17':[None,600,550],'baseline_lstm_v15':[None]}
+ Ensemble-Mean, prediction_rounded-False: 0.1452

MODEL_NAMES = {'baseline_lstm_v17':[None,600,550,500],'baseline_lstm_v15':[None]}

+ Ensemble-Median, prediction_rounded-False: 0.147524315805496
+ Ensemble-Median, prediction_rounded-True: 0.1473297930971845
+ Ensemble-Mean, prediction_rounded-True: 0.14633993365531525
+ Ensemble-Mean, prediction_rounded-False: 0.14559185437100855

MODEL_NAMES = {'baseline_lstm_v17':[None],'baseline_lstm_v15':[None]}
+ Ensemble-Median, prediction_rounded-False: 0.14574096082939075
+ Ensemble-Mean, prediction rounded-False: 0.14574096082939075
+ Ensemble-mean, prediction_rounded-True: 0.147524315805496
+ Ensemble-Median, prediction_rounded-True: 0.147524315805496 


In [146]:
test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_simple_ensemble.csv',index = False)

### 2. Ensemble with Free Kernels

In [32]:
# Ensemble with free kernel from competition
kernel_sub = pd.read_csv('kernel_subs/1336_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1348_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1353_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1355_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1357_submission.csv')
preds_list.append(kernel_sub.pressure.values)
preds_ens = np.median(np.vstack(preds_list),axis = 0)

test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_simple_ensemble_free_kernel.csv',index = False)

## Past

In [ ]:
# Ensemble with free kernel from competition
kernel_sub = pd.read_csv('kernel_subs/submission_1359.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/submission_1368.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/submission_1336.csv')
preds_list.append(kernel_sub.pressure.values)
preds_ens = np.median(np.vstack(preds_list),axis = 0)

test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_simple_ensemble_free_kernel.csv',index = False)

In [6]:
# Ensemble with free kernel from competition
preds_list = []
kernel_sub = pd.read_csv('kernel_subs/1336_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1348_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1353_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1355_submission.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/1357_submission.csv')
preds_list.append(kernel_sub.pressure.values)
preds_ens = np.median(np.vstack(preds_list),axis = 0)

test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out']]
test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('only_kernel_subs.csv',index = False)